In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
#from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler 
#from sklearn.preprocessing import RobustScaler, Normalizer, QuantileTransformer, PowerTransformer
from sklearn.model_selection import train_test_split, cross_val_score
#from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
from sklearn import preprocessing

In [2]:
df = pd.read_csv('Preprocessed_data.csv', index_col = 0)
df

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes_binary
0,0,0,1,23,0,0,0,1,0,0,...,0,1,0,0,0,0,11,5,7,0
1,1,0,1,19,0,0,0,0,1,1,...,0,3,0,0,0,0,6,6,8,0
2,0,0,1,26,1,0,0,1,1,1,...,0,2,0,0,0,0,1,4,4,0
3,0,1,1,22,0,0,0,1,1,1,...,0,1,0,0,0,1,12,4,2,0
4,0,0,1,22,0,0,0,0,1,1,...,0,1,0,0,0,0,4,6,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253659,0,1,1,37,0,0,0,0,0,1,...,0,4,0,0,0,0,6,4,1,1
253668,0,1,1,29,1,0,1,0,1,1,...,0,2,0,0,1,1,10,3,6,1
253670,1,1,1,25,0,0,1,0,1,0,...,0,5,15,0,1,0,13,6,4,1
253676,1,1,1,18,0,0,0,0,0,0,...,0,4,0,0,1,0,11,2,4,1


In [3]:
X = df.iloc[:,0:-1]
y = df.loc[:, 'Diabetes_binary']
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=10000, shuffle=True, random_state = 1)

In [4]:
acc_list = []
roc_auc_ovr_List = []
f1_micro_List = []

## Trial 1

In [5]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWar

In [6]:
best_model.cv_results_

{'mean_fit_time': array([0.01412556, 0.04239457, 0.05425239, 0.05092468, 0.07401793,
        0.05697544, 0.0591347 , 0.04778697, 0.05953901, 0.06501472,
        0.06724946, 0.0554184 , 0.07635636, 0.05205688, 0.07601104,
        0.06919959, 0.08491945, 0.06727309, 0.01940565, 0.01570559,
        0.02421415, 0.02037351, 0.01966259, 0.0199415 , 0.0190665 ,
        0.02110868, 0.02598965, 0.02459764, 0.06443098, 0.01985743]),
 'std_fit_time': array([0.00631732, 0.01076439, 0.0061579 , 0.01261764, 0.00710797,
        0.01122736, 0.00962797, 0.00702328, 0.01149314, 0.01103928,
        0.01307511, 0.01294183, 0.0184085 , 0.00733833, 0.01112508,
        0.01433921, 0.01746264, 0.01197843, 0.00623642, 0.00345271,
        0.00759256, 0.00620458, 0.00633709, 0.0075049 , 0.00667065,
        0.00703841, 0.00677995, 0.00889527, 0.00684424, 0.00598256]),
 'mean_score_time': array([0.00701127, 0.00797207, 0.00776415, 0.00399191, 0.00778275,
        0.00590842, 0.00686014, 0.00581388, 0.00738184, 0.00

In [7]:
best_model.cv_results_['rank_test_accuracy']

array([30, 27, 29, 25, 21, 23, 21, 19,  1, 16,  2,  2,  2,  2,  2,  2,  2,
        2, 27, 25, 23, 19, 16,  2,  2,  2,  2,  2,  2, 16])

In [8]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [9]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_acc'] = 1 - best_model.cv_results_['mean_test_accuracy']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_acc'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.3961
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.3082
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.3143
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.2636
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.2595
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.2599
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.2595
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.2592
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.2586
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.2590


In [10]:
acc_list.append(results[0:1])

In [11]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [12]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_precision'] = 1 - best_model.cv_results_['mean_test_precision']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_precision'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,1.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.245991
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.274630
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.296503
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.305961
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.310645
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.311650
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.311995
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.311362
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.312041


In [13]:
roc_auc_ovr_List.append(results[1:2])

In [14]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 1.0,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [15]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_f1_micro'] = 1 - best_model.cv_results_['mean_test_f1_micro']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_f1_micro'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.3961
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.3082
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.3143
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.2636
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.2595
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.2599
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.2595
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.2592
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.2586
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.2590


In [16]:
f1_micro_List.append(results[8:9])

## Trial 2

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=10000, shuffle=True, random_state = 2)

In [18]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWar

In [19]:
best_model.cv_results_

{'mean_fit_time': array([0.01931539, 0.06725841, 0.07058723, 0.07815807, 0.08596368,
        0.06513565, 0.07429421, 0.06068981, 0.09605699, 0.07257047,
        0.08978846, 0.06927693, 0.10030444, 0.06447778, 0.08035533,
        0.07719707, 0.10697193, 0.09166915, 0.01990266, 0.01997254,
        0.01551003, 0.02186108, 0.02316077, 0.02047017, 0.02221661,
        0.01846945, 0.02109344, 0.02554944, 0.08343911, 0.02281282]),
 'std_fit_time': array([0.00946069, 0.01009995, 0.01028016, 0.00815188, 0.00936993,
        0.01546043, 0.01214767, 0.01173103, 0.0145212 , 0.01300433,
        0.01672919, 0.0142411 , 0.02178602, 0.01306638, 0.01478341,
        0.01096509, 0.02272507, 0.0161696 , 0.00684441, 0.00633065,
        0.00142041, 0.00791449, 0.0071933 , 0.00713376, 0.00794164,
        0.00666907, 0.00710497, 0.00828161, 0.0190903 , 0.00740047]),
 'mean_score_time': array([0.00947127, 0.01139772, 0.00956721, 0.00591559, 0.00255997,
        0.00986736, 0.00597911, 0.00419984, 0.00924733, 0.00

In [20]:
best_model.cv_results_['rank_test_accuracy']

array([30, 27, 29, 20, 26,  1, 25, 23, 22,  1,  1,  7,  7,  7,  7, 19,  7,
        7, 27, 20,  1, 23,  1,  7,  7,  7,  7,  7,  7,  1])

In [21]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [22]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_acc'] = 1 - best_model.cv_results_['mean_test_accuracy']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_acc'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.3996
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.3083
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.3163
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.2588
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.2610
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.2586
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.2597
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.2593
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.2589
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.2586


In [23]:
acc_list.append(results[0:1])

In [24]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [25]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_precision'] = 1 - best_model.cv_results_['mean_test_precision']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_precision'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,1.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.245796
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.262296
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.294790
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.311106
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.312532
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.316062
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.315614
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.315009
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.314652


In [26]:
roc_auc_ovr_List.append(results[1:2])

In [27]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.01,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [28]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_f1_micro'] = 1 - best_model.cv_results_['mean_test_f1_micro']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_f1_micro'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.3996
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.3083
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.3163
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.2588
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.2610
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.2586
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.2597
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.2593
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.2589
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.2586


In [29]:
f1_micro_List.append(results[5:6])

## Trial 3

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=10000, shuffle=True, random_state = 3)

In [31]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWar

In [32]:
best_model.cv_results_

{'mean_fit_time': array([0.02211905, 0.08105481, 0.07513254, 0.06575971, 0.08989377,
        0.07971194, 0.09664969, 0.0853009 , 0.09677811, 0.07151542,
        0.09715207, 0.0766947 , 0.09966738, 0.08958099, 0.12130175,
        0.06307886, 0.11692622, 0.09505532, 0.02223752, 0.02380641,
        0.02312493, 0.02728791, 0.02727604, 0.02962677, 0.02674415,
        0.02199893, 0.02418122, 0.02611551, 0.07725699, 0.02045481]),
 'std_fit_time': array([0.005711  , 0.0170219 , 0.01187235, 0.00967367, 0.01801439,
        0.01366517, 0.01783743, 0.0141837 , 0.01320169, 0.0074765 ,
        0.01627375, 0.01064508, 0.01795209, 0.00631787, 0.02378282,
        0.00032192, 0.0269617 , 0.0142864 , 0.00751808, 0.00757814,
        0.00842681, 0.00783307, 0.00638048, 0.0058297 , 0.00727265,
        0.00771574, 0.00743941, 0.0071573 , 0.01493379, 0.0059865 ]),
 'mean_score_time': array([0.0092324 , 0.00633562, 0.00794089, 0.00978112, 0.01072121,
        0.00910316, 0.00872109, 0.00625455, 0.01014206, 0.00

In [33]:
best_model.cv_results_['rank_test_accuracy']

array([30, 27, 29, 25, 24,  1,  3,  3,  6, 21,  6,  6,  6,  6,  6,  6,  6,
        6, 27, 25,  1,  3, 21,  6,  6,  6,  6,  6,  6, 21])

In [34]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [35]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_acc'] = 1 - best_model.cv_results_['mean_test_accuracy']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_acc'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.3989
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.3119
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.3210
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.2575
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.2549
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.2539
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.2541
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.2541
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.2544
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.2545


In [36]:
acc_list.append(results[0:1])

In [37]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [38]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_precision'] = 1 - best_model.cv_results_['mean_test_precision']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_precision'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,1.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.248720
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.280813
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.291078
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.301133
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.302672
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.305823
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.306056
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.306629
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.306701


In [39]:
roc_auc_ovr_List.append(results[1:2])

In [40]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.01,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [41]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_f1_micro'] = 1 - best_model.cv_results_['mean_test_f1_micro']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_f1_micro'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.3989
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.3119
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.3210
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.2575
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.2549
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.2539
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.2541
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.2541
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.2544
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.2545


In [42]:
f1_micro_List.append(results[5:6])

## Trial 4

In [43]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=10000, shuffle=True, random_state = 4)

In [44]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWar

In [45]:
best_model.cv_results_

{'mean_fit_time': array([0.01858034, 0.0841594 , 0.0995791 , 0.08477128, 0.07921925,
        0.08084464, 0.09603736, 0.08845074, 0.11162031, 0.08697999,
        0.10200436, 0.07311423, 0.09666393, 0.08619237, 0.11343811,
        0.07996111, 0.09644594, 0.08088238, 0.02229652, 0.02040348,
        0.02649186, 0.02387068, 0.02358284, 0.02576334, 0.02211382,
        0.02775686, 0.02411215, 0.0282222 , 0.08803594, 0.02193959]),
 'std_fit_time': array([0.00411804, 0.01603266, 0.01450352, 0.01418431, 0.01934171,
        0.01858413, 0.01647499, 0.0143033 , 0.01898208, 0.01519758,
        0.01911303, 0.02024067, 0.01816598, 0.01091002, 0.00903352,
        0.01798353, 0.0257164 , 0.01034268, 0.00764347, 0.00722532,
        0.00759949, 0.00742728, 0.00772676, 0.00742143, 0.00759646,
        0.00626972, 0.00720665, 0.00695451, 0.01812412, 0.00680425]),
 'mean_score_time': array([0.00908272, 0.00972142, 0.00471222, 0.00937762, 0.00892999,
        0.00874228, 0.01092696, 0.00447421, 0.00627153, 0.00

In [46]:
best_model.cv_results_['rank_test_accuracy']

array([30, 27, 29, 25, 26, 21, 23, 19, 18,  1,  1,  1,  1,  1,  1,  1,  1,
        1, 27, 24, 21, 19,  1,  1,  1,  1,  1,  1,  1,  1])

In [47]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [48]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_acc'] = 1 - best_model.cv_results_['mean_test_accuracy']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_acc'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.3989
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.3035
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.3156
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.2602
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.2603
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.2582
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.2589
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.2570
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.2568
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.2567


In [49]:
acc_list.append(results[0:1])

In [50]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [51]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_precision'] = 1 - best_model.cv_results_['mean_test_precision']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_precision'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,1.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.236486
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.268945
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.296595
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.310450
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.309520
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.313062
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.310365
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.310318
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.310238


In [52]:
roc_auc_ovr_List.append(results[1:2])

In [53]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 1.0,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [54]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_f1_micro'] = 1 - best_model.cv_results_['mean_test_f1_micro']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_f1_micro'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.3989
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.3035
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.3156
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.2602
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.2603
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.2582
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.2589
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.2570
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.2568
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.2567


In [55]:
f1_micro_List.append(results[9:10])

## Trial 5

In [56]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=10000, shuffle=True, random_state = 5)

In [57]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWar

In [58]:
best_model.cv_results_

{'mean_fit_time': array([0.02308955, 0.08153121, 0.09174404, 0.07765205, 0.097838  ,
        0.09905071, 0.10192332, 0.0841186 , 0.12345746, 0.0996366 ,
        0.11807191, 0.10311475, 0.12374725, 0.10500593, 0.11119056,
        0.07990174, 0.08287246, 0.05497427, 0.01159201, 0.01191318,
        0.01442766, 0.01416473, 0.01580408, 0.01551905, 0.01591287,
        0.01684818, 0.01673398, 0.01532507, 0.06331613, 0.01323433]),
 'std_fit_time': array([0.00705376, 0.01441821, 0.00873333, 0.01881778, 0.02095909,
        0.01008592, 0.02203431, 0.01764827, 0.01106505, 0.01851002,
        0.0225421 , 0.01077031, 0.01380524, 0.01093791, 0.02982655,
        0.02165508, 0.01249654, 0.01043635, 0.00610058, 0.00622558,
        0.00489551, 0.00472476, 0.007544  , 0.0078442 , 0.00701444,
        0.00746329, 0.00298112, 0.00993653, 0.0101787 , 0.00543821]),
 'mean_score_time': array([0.00755734, 0.00777135, 0.00800302, 0.0075686 , 0.00883584,
        0.00943551, 0.00578816, 0.00687647, 0.00853822, 0.00

In [59]:
best_model.cv_results_['rank_test_accuracy']

array([30, 27, 29, 25, 24, 22,  2, 20,  1, 17,  2,  2,  2,  2,  2,  2,  2,
        2, 27, 25, 22, 20, 17,  2,  2,  2,  2,  2,  2, 17])

In [60]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [61]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_acc'] = 1 - best_model.cv_results_['mean_test_accuracy']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_acc'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.4021
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.3042
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.3149
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.2617
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.2566
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.2551
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.2532
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.2535
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.2531
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.2533


In [62]:
acc_list.append(results[0:1])

In [63]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [64]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_precision'] = 1 - best_model.cv_results_['mean_test_precision']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_precision'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,1.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.243476
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.259802
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.297109
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.301208
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.302437
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.301160
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.302370
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.301828
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.302316


In [65]:
roc_auc_ovr_List.append(results[1:2])

In [66]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 1.0,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [67]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_f1_micro'] = 1 - best_model.cv_results_['mean_test_f1_micro']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_f1_micro'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.4021
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.3042
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.3149
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.2617
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.2566
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.2551
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.2532
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.2535
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.2531
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.2533


In [68]:
f1_micro_List.append(results[6:7])

## Trial 6

In [69]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=10000, shuffle=True, random_state = 6)

In [70]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWar

In [71]:
best_model.cv_results_

{'mean_fit_time': array([0.01737266, 0.04388099, 0.06550457, 0.05065863, 0.06950455,
        0.0529268 , 0.07073174, 0.04736278, 0.06015787, 0.0656549 ,
        0.06823819, 0.04518266, 0.07253709, 0.05298262, 0.0795907 ,
        0.04823523, 0.06614304, 0.04614034, 0.01425509, 0.01751101,
        0.01432757, 0.01526265, 0.0152184 , 0.01628709, 0.01428044,
        0.01482353, 0.01339905, 0.01592877, 0.05461521, 0.01631558]),
 'std_fit_time': array([0.00472452, 0.01103916, 0.0150384 , 0.01371228, 0.01028058,
        0.00937371, 0.01233207, 0.00983137, 0.01622413, 0.00883396,
        0.00991749, 0.01083791, 0.01612537, 0.00680829, 0.01088086,
        0.00938027, 0.01152216, 0.00496467, 0.00535707, 0.00421253,
        0.00703245, 0.00726437, 0.00644241, 0.00547372, 0.00929048,
        0.00334867, 0.004935  , 0.0071696 , 0.01089254, 0.00729176]),
 'mean_score_time': array([0.00716038, 0.01086452, 0.0052675 , 0.00591424, 0.00627882,
        0.00686533, 0.00439818, 0.00654185, 0.00155678, 0.00

In [72]:
best_model.cv_results_['rank_test_accuracy']

array([30, 27, 29, 25, 24, 12,  1, 22, 21,  2,  2,  2, 14, 14, 14, 14, 14,
       14, 27, 25, 12, 22,  2,  2,  2,  2,  2,  2, 14,  2])

In [73]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [74]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_acc'] = 1 - best_model.cv_results_['mean_test_accuracy']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_acc'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.3997
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.2966
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.3059
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.2538
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.2489
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.2462
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.2460
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.2467
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.2465
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.2462


In [75]:
acc_list.append(results[0:1])

In [76]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [77]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_precision'] = 1 - best_model.cv_results_['mean_test_precision']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_precision'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,1.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.221245
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.243304
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.287826
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.293287
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.293150
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.294969
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.295892
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.296044
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.295616


In [78]:
roc_auc_ovr_List.append(results[1:2])

In [79]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.1,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [80]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_f1_micro'] = 1 - best_model.cv_results_['mean_test_f1_micro']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_f1_micro'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.3997
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.2966
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.3059
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.2538
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.2489
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.2462
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.2460
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.2467
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.2465
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.2462


In [81]:
f1_micro_List.append(results[6:7])

## Trial 7

In [82]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=10000, shuffle=True, random_state = 7)

In [83]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWar

In [84]:
best_model.cv_results_

{'mean_fit_time': array([0.01452353, 0.0551595 , 0.06368232, 0.04734917, 0.07077663,
        0.06042213, 0.05941138, 0.05519729, 0.0742033 , 0.07545686,
        0.0682111 , 0.06295273, 0.07559428, 0.06773322, 0.0816906 ,
        0.05791404, 0.07374501, 0.05019147, 0.0112505 , 0.01309164,
        0.01345232, 0.01732965, 0.01458604, 0.01585441, 0.01480513,
        0.0165906 , 0.01589704, 0.01734266, 0.05895212, 0.01252215]),
 'std_fit_time': array([0.00666532, 0.01266862, 0.01200235, 0.00688629, 0.01619007,
        0.01237139, 0.01358514, 0.00964634, 0.01455343, 0.01112782,
        0.02212531, 0.01402939, 0.01835173, 0.00939087, 0.02084697,
        0.01244084, 0.01336013, 0.00938906, 0.00649839, 0.00477087,
        0.00483802, 0.00477827, 0.00512133, 0.00700503, 0.00415294,
        0.00760235, 0.00402348, 0.0055061 , 0.01258054, 0.00718736]),
 'mean_score_time': array([0.00444281, 0.00622821, 0.00946808, 0.00627441, 0.00624731,
        0.00261033, 0.00666175, 0.00762055, 0.01047239, 0.00

In [85]:
best_model.cv_results_['rank_test_accuracy']

array([30, 27, 29, 25, 24, 22,  2,  3,  1,  3,  3,  3,  3,  3,  3,  3,  3,
        3, 27, 25, 22,  3,  3,  3,  3,  3,  3,  3,  3,  3])

In [86]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [87]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_acc'] = 1 - best_model.cv_results_['mean_test_accuracy']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_acc'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.4034
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.3052
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.3172
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.2600
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.2548
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.2524
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.2521
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.2522
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.2520
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.2522


In [88]:
acc_list.append(results[0:1])

In [89]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [90]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_precision'] = 1 - best_model.cv_results_['mean_test_precision']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_precision'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,1.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.238699
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.269254
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.293224
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.300284
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.298798
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.300779
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.300493
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.300441
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.300690


In [91]:
roc_auc_ovr_List.append(results[1:2])

In [92]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 1.0,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [93]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_f1_micro'] = 1 - best_model.cv_results_['mean_test_f1_micro']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_f1_micro'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.4034
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.3052
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.3172
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.2600
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.2548
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.2524
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.2521
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.2522
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.2520
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.2522


In [94]:
f1_micro_List.append(results[8:9])

## Trial 8

In [95]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=10000, shuffle=True, random_state = 8)

In [96]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWar

In [97]:
best_model.cv_results_

{'mean_fit_time': array([0.01573954, 0.04967554, 0.05961335, 0.04956481, 0.06310413,
        0.05118489, 0.06650667, 0.0556365 , 0.06461713, 0.06610632,
        0.05824928, 0.04425726, 0.04875224, 0.04719474, 0.05545895,
        0.04992056, 0.06162398, 0.04736364, 0.01355455, 0.01496539,
        0.01490083, 0.01364672, 0.01300013, 0.01553354, 0.01287289,
        0.01052482, 0.01677923, 0.01535447, 0.04790475, 0.01499112]),
 'std_fit_time': array([0.00693178, 0.00866263, 0.01341329, 0.00691133, 0.01190741,
        0.01169518, 0.00994149, 0.01164398, 0.00875937, 0.01168523,
        0.00748653, 0.00564171, 0.00320547, 0.00024357, 0.00737897,
        0.00797433, 0.00362978, 0.00407446, 0.00465534, 0.00231565,
        0.00553434, 0.00760383, 0.00526596, 0.00421368, 0.00571954,
        0.00702524, 0.00322277, 0.00589278, 0.00806003, 0.00282629]),
 'mean_score_time': array([0.00797052, 0.005515  , 0.00646329, 0.00703733, 0.00471327,
        0.00702324, 0.00783663, 0.00417101, 0.00844791, 0.00

In [98]:
best_model.cv_results_['rank_test_accuracy']

array([30, 27, 29, 25, 24,  2,  1,  5,  4,  5, 10, 10, 10, 10, 10, 10, 10,
       10, 27, 25,  2,  5,  5, 10, 10, 10, 10, 10, 10,  5])

In [99]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [100]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_acc'] = 1 - best_model.cv_results_['mean_test_accuracy']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_acc'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.4069
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.3008
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.3135
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.2483
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.2471
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.2463
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.2453
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.2464
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.2464
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.2464


In [101]:
acc_list.append(results[0:1])

In [102]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [103]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_precision'] = 1 - best_model.cv_results_['mean_test_precision']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_precision'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,1.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.235141
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.248472
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.278156
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.288445
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.291108
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.291849
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.293329
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.293319
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.293245


In [104]:
roc_auc_ovr_List.append(results[1:2])

In [105]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.1,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [106]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_f1_micro'] = 1 - best_model.cv_results_['mean_test_f1_micro']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_f1_micro'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.4069
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.3008
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.3135
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.2483
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.2471
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.2463
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.2453
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.2464
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.2464
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.2464


In [107]:
f1_micro_List.append(results[6:7])

## Trial 9

In [108]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=10000, shuffle=True, random_state = 9)

In [109]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWar

In [110]:
best_model.cv_results_

{'mean_fit_time': array([0.00856683, 0.04491405, 0.04667151, 0.04508078, 0.0516027 ,
        0.04677253, 0.05672667, 0.045591  , 0.05495868, 0.04786475,
        0.0567843 , 0.04507842, 0.05697658, 0.04800336, 0.0538439 ,
        0.0458421 , 0.05170581, 0.04254258, 0.0126024 , 0.01133897,
        0.01363463, 0.01582949, 0.01525238, 0.01429677, 0.01297162,
        0.01417441, 0.01507125, 0.01686378, 0.04887755, 0.01518228]),
 'std_fit_time': array([0.0066089 , 0.00524514, 0.00382878, 0.00872046, 0.00737454,
        0.00116615, 0.00730341, 0.00478683, 0.0086961 , 0.00963725,
        0.00831757, 0.00479615, 0.00731273, 0.0026454 , 0.00753353,
        0.0038575 , 0.00605455, 0.00696722, 0.00690682, 0.00662506,
        0.00749616, 0.00404057, 0.00144201, 0.00480595, 0.00761546,
        0.00472873, 0.00184831, 0.00797889, 0.0041788 , 0.00581332]),
 'mean_score_time': array([0.01211023, 0.00164735, 0.00847194, 0.00771332, 0.00425959,
        0.00824516, 0.00315061, 0.0093749 , 0.00472288, 0.00

In [111]:
best_model.cv_results_['rank_test_accuracy']

array([30, 27, 29, 25,  1,  2, 24,  4,  6, 20, 20,  7,  7,  7,  7,  7,  7,
        7, 27, 25,  2,  4, 20,  7,  7,  7,  7,  7,  7, 20])

In [112]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [113]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_acc'] = 1 - best_model.cv_results_['mean_test_accuracy']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_acc'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.3954
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.3133
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.3168
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.2568
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.2533
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.2544
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.2552
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.2548
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.2549
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.2550


In [114]:
acc_list.append(results[0:1])

In [115]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [116]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_precision'] = 1 - best_model.cv_results_['mean_test_precision']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_precision'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,1.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.241676
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.274449
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.290506
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.301099
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.305489
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.307907
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.307538
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.307743
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.308028


In [117]:
roc_auc_ovr_List.append(results[1:2])

In [118]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.01,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [119]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_f1_micro'] = 1 - best_model.cv_results_['mean_test_f1_micro']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_f1_micro'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.3954
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.3133
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.3168
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.2568
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.2533
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.2544
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.2552
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.2548
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.2549
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.2550


In [120]:
f1_micro_List.append(results[4:5])

## Trial 10

In [121]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=10000, shuffle=True, random_state = 10)

In [122]:
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', LogisticRegression())])

# Create search space of candidate learning algorithms and their hyperparameters
# note lbfgs can't do l1, and if you pass penalty='none' it expects no C value
search_space = [{'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [LogisticRegression(max_iter=10000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']},
                {'classifier': [LogisticRegression(max_iter=10000)]}
                ]

# Create grid search
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10),
                   scoring=['accuracy', 'precision', 'f1_micro'], refit=False,
                   verbose=0)

# Fit grid search
best_model = clf.fit(X_train, Y_train)

C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guest-All\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWar

In [123]:
best_model.cv_results_

{'mean_fit_time': array([0.01226552, 0.04000165, 0.05454657, 0.0484731 , 0.05518978,
        0.04393113, 0.0578877 , 0.04768958, 0.0621753 , 0.0531491 ,
        0.06601825, 0.04763103, 0.06237292, 0.04940424, 0.05939221,
        0.04988   , 0.05814936, 0.04569244, 0.01320791, 0.01715674,
        0.01999102, 0.01500506, 0.0152216 , 0.01494484, 0.01149192,
        0.01627951, 0.01571286, 0.00930028, 0.04674072, 0.01411378]),
 'std_fit_time': array([0.00591601, 0.01070684, 0.00752878, 0.00487624, 0.00771433,
        0.00742728, 0.00702765, 0.00307363, 0.00902242, 0.0074868 ,
        0.00624278, 0.0072178 , 0.00719747, 0.00533581, 0.00548405,
        0.00751011, 0.00726871, 0.00468186, 0.00527247, 0.00591886,
        0.00771682, 0.00175927, 0.00611885, 0.00214115, 0.00655682,
        0.00736244, 0.00015114, 0.00691243, 0.00737582, 0.00471132]),
 'mean_score_time': array([0.00788662, 0.00633593, 0.00782175, 0.00649297, 0.00605037,
        0.00641944, 0.00496347, 0.00413256, 0.0033463 , 0.00

In [124]:
best_model.cv_results_['rank_test_accuracy']

array([30, 27, 29, 25, 24, 22, 21, 10, 10,  1, 10,  1, 10,  1,  1,  1, 10,
        1, 27, 25, 22, 10,  1, 10, 10, 10, 10, 10,  1,  1])

In [125]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['mean_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [126]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_acc'] = 1 - best_model.cv_results_['mean_test_accuracy']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_acc'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.3969
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.3131
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.3156
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.2645
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.2588
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.2566
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.2564
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.2561
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.2561
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.2560


In [127]:
acc_list.append(results[0:1])

In [128]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_precision'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [129]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_precision'] = 1 - best_model.cv_results_['mean_test_precision']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_precision'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_precision
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,1.000000
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.247350
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.275230
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.298281
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.304219
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.305584
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.306775
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.306418
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.306387
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.306304


In [130]:
roc_auc_ovr_List.append(results[1:2])

In [131]:
best_model.cv_results_['params'][ np.argmin(best_model.cv_results_['rank_test_f1_micro'])]

{'classifier': LogisticRegression(max_iter=10000),
 'classifier__C': 1.0,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [132]:
results = pd.DataFrame( best_model.cv_results_['params'] )

# next grab the score resulting from those parameters, add it to the data
# score is accuracy; to display it as misclassification error we use 1 - x
results['score_f1_micro'] = 1 - best_model.cv_results_['mean_test_f1_micro']

# get rid of classifierXX in columns
cols = results.columns.to_series().str.split('__').apply(lambda x: x[-1])
results.columns = cols

# lets show the results for the saga solver across penalty & C values
#sns.heatmap( results.query('solver=="saga"').pivot('C','penalty','score_f1_micro'),
#             annot=True, fmt='.3f')

results

,classifier,C,penalty,solver,score_f1_micro
0,LogisticRegression(max_iter=10000),0.0001,l1,saga,0.3969
1,LogisticRegression(max_iter=10000),0.0001,l2,saga,0.3131
2,LogisticRegression(max_iter=10000),0.0010,l1,saga,0.3156
3,LogisticRegression(max_iter=10000),0.0010,l2,saga,0.2645
4,LogisticRegression(max_iter=10000),0.0100,l1,saga,0.2588
5,LogisticRegression(max_iter=10000),0.0100,l2,saga,0.2566
6,LogisticRegression(max_iter=10000),0.1000,l1,saga,0.2564
7,LogisticRegression(max_iter=10000),0.1000,l2,saga,0.2561
8,LogisticRegression(max_iter=10000),1.0000,l1,saga,0.2561
9,LogisticRegression(max_iter=10000),1.0000,l2,saga,0.2560


In [133]:
f1_micro_List.append(results[9:10])

## Results

In [193]:
accuracy = [x['score_acc'][0] for x in acc_list]
accuracy_c = [x['C'][0] for x in acc_list]
accuracy_penalty = [x['penalty'][0] for x in acc_list]
roc = [x['score_precision'].reset_index(drop=True)[0] for x in roc_auc_ovr_List]
roc_c = [x['C'].reset_index(drop=True)[0] for x in roc_auc_ovr_List]
roc_penalty = [x['penalty'].reset_index(drop=True)[0] for x in roc_auc_ovr_List]
f1 = [x['score_f1_micro'].reset_index(drop=True)[0] for x in f1_micro_List]
f1_c = [x['C'].reset_index(drop=True)[0] for x in f1_micro_List]
f1_penalty = [x['penalty'].reset_index(drop=True)[0] for x in f1_micro_List]

In [194]:
data = {'Accuracy': accuracy, 'Accuracy C': accuracy_c, 'Accuracy Penalty': accuracy_penalty,
        'Roc_auc_ovr': roc, 'Roc_auc_ovr C': roc_c, 'Roc_auc_ovr Penalty': roc_penalty,
        'F1_micro':f1, 'F1_micro C': f1_c, 'F1_micro Penalty': f1_penalty}

In [195]:
trainingResults  = pd.DataFrame(data = data)
pd.options.display.max_colwidth = 100
trainingResults

,Accuracy,Accuracy C,Accuracy Penalty,Roc_auc_ovr,Roc_auc_ovr C,Roc_auc_ovr Penalty,F1_micro,F1_micro C,F1_micro Penalty
0,0.3961,0.0001,l1,0.245991,0.0001,l2,0.2586,1.00,l1
1,0.3996,0.0001,l1,0.245796,0.0001,l2,0.2586,0.01,l2
2,0.3989,0.0001,l1,0.248720,0.0001,l2,0.2539,0.01,l2
3,0.3989,0.0001,l1,0.236486,0.0001,l2,0.2567,1.00,l2
4,0.4021,0.0001,l1,0.243476,0.0001,l2,0.2532,0.10,l1
5,0.3997,0.0001,l1,0.221245,0.0001,l2,0.2460,0.10,l1
6,0.4034,0.0001,l1,0.238699,0.0001,l2,0.2520,1.00,l1
7,0.4069,0.0001,l1,0.235141,0.0001,l2,0.2453,0.10,l1
8,0.3954,0.0001,l1,0.241676,0.0001,l2,0.2533,0.01,l1
9,0.3969,0.0001,l1,0.247350,0.0001,l2,0.2560,1.00,l2


In [196]:
trainingResults.to_csv('LR_trainingResults.csv')